In [1]:
%load_ext autoreload
%autoreload 2
%pwd

'/home/luca/uni/master/dl-in-vc/voxl3r'

In [2]:
# setup PYTHONPATH
import sys
sys.path += ['.', './extern/scannetpp', './extern/mast3r', './extern/mast3r/dust3r']

In [3]:
from pathlib import Path
import torch
from torchvision.io import read_image

import pyvista as pv
pv.set_jupyter_backend('client')

In [4]:
pv.start_xvfb()

In [9]:
from models.surface_net_3d.run import visualize_run

run_name = "0kvmt93e"

visualize_run(run_name)

No best_ckpts.pt found, falling back to last checkpoint
No data_config found in checkpoint, using default
First image of training is: /home/luca/mnt/data/scannetpp/data/0cf2e9402d/iphone/rgb/frame_006860.jpg
Chunk size was [1.28 1.28 1.28], resolution was 0.02, center was [0.         0.         1.47199988]
Grid size was 64
First image of validation is: /home/luca/mnt/data/scannetpp/data/0cf2e9402d/iphone/rgb/frame_004590.jpg
Chunk size was [1.28 1.28 1.28], resolution was 0.02, center was [0.         0.         1.47199988]
First image of test is: /home/luca/mnt/data/scannetpp/data/0cf2e9402d/iphone/rgb/frame_006820.jpg
Chunk size was [1.28 1.28 1.28], resolution was 0.02, center was [0.         0.         1.47199988]


Widget(value='<iframe src="http://localhost:41685/index.html?ui=P_0x74afdc555ee0_4&reconnect=auto" class="pyvi…

AttributeError: 
This plotter has not yet been set up and rendered with ``show()``.
Consider setting ``off_screen=True`` for off screen rendering.


In [ ]:
import numpy as np
from dataset import SceneDataset
from models.surface_net_3d.data import SurfaceNet3DDataConfig
from utils.chunking import create_chunk, mesh_2_voxels
from utils.data_parsing import load_yaml_munch
from utils.transformations import invert_pose
from utils.visualize import visualize_mesh

config = load_yaml_munch("./utils/config.yaml")
data_config = SurfaceNet3DDataConfig(data_dir=config.data_dir, batch_size=2)
resolution = data_config.grid_resolution
grid_size = data_config.grid_size
max_seq_len = data_config.seq_len

resolution = 0.02
chunk_size = resolution * grid_size.astype(np.float32)
center = np.array([0.0, 0.0, chunk_size[2]])

dataset = SceneDataset(
            camera=data_config.camera,
            data_dir=data_config.data_dir,
            scenes=data_config.scenes,
        )

data = dataset[0]
mesh = data["mesh"]
path_images = data["path_images"]
camera_params = data["camera_params"]

image_name = "frame_000860.jpg"

data_chunk = create_chunk(mesh.copy(), image_name, camera_params, max_seq_len=max_seq_len, image_path=path_images, center=center, size=chunk_size, with_backtransform=True)

T_cw = camera_params[image_name]['T_cw']
_, _, T_wc = invert_pose(T_cw[:3,:3], T_cw[:3,3])

mesh_chunk_backtransformer = data_chunk["backtransformed"]
mesh_chunk = data_chunk["mesh"]
image_names_chunk = data_chunk["image_names"]
camera_params_chunk = data_chunk["camera_params"]
p_center = data_chunk["p_center"]

print(f"Chunk size: {chunk_size}")
print(f"Center: {center}")
print(f"Resolution: {resolution}")

In [ ]:
data["camera_params"][image_name]['K']

In [ ]:
voxel_grid, coordinates, occupancy_values = mesh_2_voxels(mesh_chunk, voxel_size=0.02, to_world_coordinates=T_wc)
mesh_chunk_backtransformer.extents
visualize_mesh(voxel_grid.as_boxes(), images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)

In [ ]:
from utils.chunking import mesh_2_local_voxels
from utils.transformations import invert_pose

voxel_grid_local, coordinates_local, occupancy_values_local  = mesh_2_local_voxels(mesh_chunk_backtransformer,  center, 0.02, grid_size[0])
visualize_mesh(voxel_grid_local.as_boxes(), images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)

In [ ]:
from utils.chunking import mesh_2_local_voxels
from utils.transformations import invert_pose


T_cw = camera_params[image_name]['T_cw']
_, _, T_wc = invert_pose(T_cw[:3,:3], T_cw[:3,3])

voxel_grid_local, coordinates_local, occupancy_values_local  = mesh_2_local_voxels(mesh_chunk,  center, 0.02, grid_size[0])
visualize_mesh(voxel_grid_local.as_boxes(), images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)